## PREDICCIÓN DE TEXTO

In [2]:
import cohere
import os
from dotenv import load_dotenv

load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")

if not cohere_api_key:
    raise ValueError("Qué onda la API_KEY. Please set the COHERE_API_KEY environment variable.")

co = cohere.Client(cohere_api_key)

try:
    response = co.generate(
        model='command-xlarge-nightly',  
        prompt="Continúa la siguiente frase de manera creativa y fluida: "+
            "Un día, soy de tierra, me conecto con la realidad desde el "+
            "mantenimiento del orden y la rutina. Muy ocasionalmente, "+
            "soy aire, me vuelvo a encontrar con...",
        max_tokens=80,
        temperature=0.9
    )

    print(response.generations[0].text)

except cohere.CohereAPIError as e:
    print(f"An error occurred: {e}")

Un día, soy de tierra, me conecto con la realidad a través del orden y la rutina que me mantienen anclado. Pero muy ocasionalmente, me transformo en aire, me elevo por encima de las limitaciones terrenales y me vuelvo a encontrar con mi espíritu aventurero. Dejo que mi imaginación vuele, explorando mundos lejanos y nuevas posibilidades. Me


## Plantillas de prompt

In [3]:
templates = {
   "historia": ("Continúa la siguiente historia de manera creativa y fluida: "
                 "Había una vez un {character} en un {place}, "
                 "que estaba {activity}. Un día, {event}. "
                 "{question}")
}

selected_template = templates["historia"]
prompt = selected_template.format(character="bravo trabajador", 
                                  place="país distópico", 
                                  activity="rompiéndose el lomo", 
                                  event="descubrió que estaba siendo estafado",
                                  question="¿Tiene final feliz?")


response = co.generate(
        model='command-xlarge-nightly',  
        prompt=prompt,
        max_tokens=80,
        temperature=0.7
    )
print(response.generations[0].text)

Había una vez un bravo trabajador en un país distópico, que se rompía el lomo día tras día en una fábrica opresiva. Su sudor y esfuerzo parecían no tener fin, y a menudo se preguntaba si su arduo trabajo realmente valía la pena.

Un día, mientras revisaba su cheque de pago, se dio cuenta de que algo and


### Parámetros (temperatura, k y p)

In [3]:
response = co.generate(
        model='command-xlarge-nightly',  
        prompt='Name a country from Oceania',
        max_tokens=5,
        temperature=0.1,
        k=5,
        p=0.9
    )
print(response.generations[0].text)

## LangSmith

In [4]:
langsmith_api_key = os.getenv("LANGSMITH_API_KEY")

from langchain_cohere import ChatCohere

model = ChatCohere(model="command-r-plus")

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="goodbye!"),
]

model.invoke(messages)

AIMessage(content='Arrivederci!', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2d4bede1-f9d7-48cd-a71d-e61ade4ed494', 'token_count': {'input_tokens': 79, 'output_tokens': 4}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '2d4bede1-f9d7-48cd-a71d-e61ade4ed494', 'token_count': {'input_tokens': 79, 'output_tokens': 4}}, id='run-17f8117f-4609-4390-95cd-935e8b61efcf-0', usage_metadata={'input_tokens': 79, 'output_tokens': 4, 'total_tokens': 83})

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
result = model.invoke(messages)
parser.invoke(result)

'Arrivederci!'

### prompt-templates

In [6]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

result = prompt_template.invoke({"language": "french", "text": "cigüeña"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:'), HumanMessage(content='cigüeña')])

In [7]:
chain = prompt_template | model | parser

chain.invoke({"language": "french", "text": "cigüeña"})

'La traduction de "cigüeña" en français est "cigogne".'